# Optimizing Python with Cython: Part I

Python's key advantages are its flexibility, large number of freely available libaries, and low development overhead. Its main disadvantage is speed, which is generally comparable to what you can expect from MATLAB. Since computers are fast these days, execution speed often doesn't matter very much compared to development time. However, in the case that you're writing a performance-intensive code where speed is critical, you should certainly not be using Python.

Sometimes a code's execution is dominated by just a few performance-intensive tasks, while the rest of the program performs utility-type operations that Python excels at. In this case, the extra development overhead incurred by using a language like C++ doesn't always make sense. Instead, it's preferrable to develop the program as a whole in Python, and optimize out the few performance-intensive portions to C, C++, or FORTRAN -- since Python is written in C, it's these first two that tend to work the best. There are a variety of ways to go about this (an <a href="https://www.paypal-engineering.com/2016/09/22/python-by-the-c-side/" target="_blank">overview</a> from 2016 can be found on the Paypal engineering blog) but this notebook is going to focus on Cython specifically.

Cython is <a href="http://docs.cython.org/en/latest/src/quickstart/overview.html" target="_blank">described</a> as a "superset" of the Python language which allows easy integration of C extensions into base Python code. It uses a purpose-built compiler to translate Cython into C code, which can then be compiled and imported as a standard C extension for Python. Note that there's nothing stopping you from simply writing C extensions yourself; Python is intended to allow this functionality. Cython just makes it simpler.

Aside from providing a nice speedup, Cython also allows you to bring in C libraries with a minimum amount of hassle. It has support for multi-threading, but I get the sense that writing codes which scale well with large numbers of processors is ultimately going to end badly, and goes past the intended use case of Cython. It's not something I've examined in detail, though.

This is intended to be Part I of a series of tutorials on Cython that will be written as time permits. In this installment, I'll go through a simple test case and then provide some pointers on how to actually compile Cython code outside of the Jupyter environment that I'm using here.

First, I'll point out that I ran into some immediate trouble compiling, as Cython was unable to find a required "vcvarsall.bat" file. (This was using the Python 3 installation of Anaconda). Fortunately, I was not alone in this problem; a general explanation of the issue is provided by <a href="https://blogs.msdn.microsoft.com/pythonengineering/2016/04/11/unable-to-find-vcvarsall-bat/" target="_blank">Microsoft</a> with a specific Cython-related explanation given at the <a href="https://www.ibm.com/developerworks/community/blogs/jfp/entry/Installing_Cython_On_Anaconda_On_Windows?lang=en" target="_blank">IBM Community</a> website. The bottom line is that the problem can be rectified by installing Visual C++ Build Tools 2015, available for <a href="http://go.microsoft.com/fwlink/?LinkId=691126" target="_blank">download</a> from Microsoft. This fixed the issue.

## Some Useful Links

<a href="http://docs.cython.org/en/latest/src/quickstart/cythonize.html" target="_blank">Cython Quickstart Guide</a><br>
<a href="http://docs.cython.org/en/latest/src/quickstart/build.html" target="_blank">Cython Quick Build Guide</a><br>
<a href="https://github.com/rkern/line_profiler" target="_blank">Line Profile Tool; Profile Before you Optimize!</a><br>
<a href="http://conference.scipy.org/static/wiki/seljebotn_cython.pdf" target="_blank">Cython Overview from SciPy Conference</a><br>
<a href="http://cython.readthedocs.io/en/latest/src/tutorial/numpy.html" target="_blank">Cython Tutorial for NumPy Users</a><br>
<a href="https://www.ibm.com/developerworks/community/blogs/jfp/entry/Installing_Cython_On_Anaconda_On_Windows?lang=en" target="_blank">Explanation of vcvarsall.bat Issue with Anaconda</a><br>
<a href="http://go.microsoft.com/fwlink/?LinkId=691126" target="_blank">Visual C++ Build Tools 2015 [Download]</a><br>

# A Test Case

For now, we'll use a modified example taken from the <a href="http://docs.cython.org/en/latest/src/quickstart/cythonize.html" target="_blank">Cython Quickstart Guide</a>. I've changed their example function to use some slightly more costly trig functions, and also made sure to compare Python and Cython with a vectorized Numpy implementation, which provides a nice speedup without any fancy compilation. We'll numerically evaluate the integral

$$
J = \int_a^b \sin(x)\cos(x)dx
$$

over the interval $0$ to $\pi/2$, which evaluates to $J = 0.5$. In base Python, this is performed using a **for** loop across a set of $N$ bins. In NumPy, the numerical integration can be easily vectorized. Both versions are implemented and timed below for $N = 500000$ bins. (Note the use of the built-in Jupyter "timeit" function to evaluate the runtime of each version).





In [1]:
# Base Python with Numpy
import numpy as np
import math

# Functions to be evaluated
def fpy(x):
    return math.sin(x)*math.cos(x)

def fnp(x):
    return np.sin(x)*np.cos(x)

# Functions which perform integration
def integrate_python(a, b, N):
    s = 0
    dx = (b-a)/N
    for i in range(N):
        s += fpy(a+i*dx)
    return s*dx

def integrate_numpy(a, b, N):
    x = np.linspace(a, b, N + 1)
    dx = (b - a)/N
    return dx*np.sum(fnp(x)[:-1])

# Convenience function to display "timeit" results
def evalTime(x, name, ref = False):
    
    string = '%s: %.1f ms' % (name, x.best*1000)
    if ref:
        string += ' (%.1fx faster than base Python)' % (ref/x.best)
    print(string)

# Values for test case
a = 0.0
b = 0.5*np.pi
N = 500000

# Make sure integrations work properly
testVal1 = integrate_python(a, b, N)
testVal2 = integrate_numpy(a, b, N)

print('Base Python returns an integration of %.6f (exact answer is 0.5)' % testVal1)
print('Vectorized Numpy returns an integration of %.6f (exact answer is 0.5)' % testVal2)

# Compare evaluation time for each version of the integration function
y = %timeit -n 3 -o -q integrate_python(a, b, N)    
evalTime(y, 'Base Python')
x = %timeit -n 3 -o -q integrate_numpy(a, b, N)    
evalTime(x, 'Vectorized Numpy', ref = y.best)


Base Python returns an integration of 0.500000 (exact answer is 0.5)
Vectorized Numpy returns an integration of 0.500000 (exact answer is 0.5)
Base Python: 253.8 ms
Vectorized Numpy: 11.7 ms (21.7x faster than base Python)


As expected, the vectorized version does much better than raw Python with a **for** loop, achieving a **~22x** speedup. Now let's see what Cython can do. In the Jupyter notebook environment, a special "magic" call **%load_ext Cython** must be evaluated prior to compiling or running any Cython code.

In [2]:
# Use this command to load up the extension for use in a Jupyter notebook
%load_ext Cython

It's possible to simply compile basic Python code without alteration, but from what I've seen so far that will actually result in slower execution. Instead, we can make some relatively minor modifications to the existing Python code in order to speed it up. The most obvious thing to do is add static type declarations. When using Numpy we can get very specific about numerical types, but for now we'll just stick with **int** and **double**. We aren't assigning types to function outputs yet, only to the inputs and local variables.

In [3]:
%%cython

import math
# Now let's make some key type declarations in order to obtain an additional speedup.
def f(double x):
    return math.sin(x)*math.cos(x)


def integrate_cython_types(double a, double b, int N): # Add types to function signature
    cdef int i # Declare using "cdef"
    cdef double s, dx # Declare using "cdef"
    
    s = 0
    dx = (b - a)/N
    for i in range(N):
        s += f(a+i*dx)
    return s*dx

# Test the function. Note that Cython creates its own environment so we have to redefine these variables.
a = 0
b = 0.5*math.pi
N = 500000
testVal = integrate_cython_types(a, b, N)
print('Returns an integration of %.6f (exact answer is 0.5)' % testVal)


Returns an integration of 0.500000 (exact answer is 0.5)


In [4]:
x = %timeit -n 3 -o -q integrate_cython_types(a, b, N)    
evalTime(x, 'Cython with Types', ref = y.best)

Cython with Types: 151.6 ms (1.7x faster than base Python)


This results in a not-very-impressive speedup of **~1.7x** over base Python. 

The next step is to replace those expensive trig functions with their counterparts from the libc math library. Using Cython, these can be imported just as you would any other module:

In [5]:
%%cython

import numpy as np
from libc.math cimport sin, cos, pi
# In addition to type declarations, replace numpy math functions with libc math.
def f(double x):
    return sin(x)*cos(x)


def integrate_cython_libc(double a, double b, int N): # Add types to function signature
    cdef int i # Declare using "cdef"
    cdef double s, dx # Declare using "cdef"
    
    s = 0
    dx = (b-a)/N
    for i in range(N):
        s += f(a+i*dx)
    return s*dx

# Test the function
a = 0
b = 0.5*pi
N = 500000
testVal = integrate_cython_libc(a, b, N)
print('Returns an integration of %.6f (exact answer is 0.5)' % testVal)


Returns an integration of 0.500000 (exact answer is 0.5)


In [6]:
x = %timeit -n 3 -o -q integrate_cython_libc(a, b, N)    
evalTime(x, 'Cython with libc', ref = y.best)

Cython with libc: 30.9 ms (8.2x faster than base Python)


Now we're getting somewhere: Replacing the function calls results in an **~8x** speedup. We're still quite a bit slower than just Numpy, however. The next step is to tell Cython to treat the repeatedly evaluated function **f** as a C function by declaring it with **cdef** and assigning a type to its output. An exception handling statement must also be added to any function which uses **cdef** -- see the actual Cython documentation for an explanation of how that works. Using **except* ** is a generally-applicable solution that comes with a very slight overhead.

Function defined with **cdef** can only be called from within the compiled Cython environment, so we use regular **def** on the integration function which is called in the cell below. **cpdef** allows a function to be defined as C-style while still being called by other Python functions, but isn't necessary for use here. 

In [7]:
%%cython

# In addition to type declarations, replace the numpy calls with calls to C standard library.
import cython
from libc.math cimport sin, cos, pi

# Declare as a C function (cdef)
cdef double f(double x) except*:  # Add types to function signature
    return sin(x)*cos(x)

def integrate_cython_cfun(double a, double b, int N):  # Add types to function signature
    cdef int i  # Declare using "cdef"
    cdef double s, dx  # Declare using "cdef"
    
    s = 0
    dx = (b-a)/N
    for i in range(N):
        s += f(a+i*dx)
    return s * dx
    
# Test the function
a = 0
b = 0.5*pi
N = 500000
testVal = integrate_cython_cfun(a, b, N)
print('Returns an integration of %.6f (exact answer is 0.5)' % testVal)

Returns an integration of 0.500000 (exact answer is 0.5)


In [8]:
x = %timeit -n 3 -o -q integrate_cython_cfun(a, b, N)    
evalTime(x, 'Cython with C functions', ref = y.best)

Cython with C functions: 7.2 ms (35.2x faster than base Python)


That's more like it: With a full set of optimization in place, the Cython version can finally beat Numpy, with a speedup of **~35x**. Some additional optimization flags are also available -- disabling various checks such as bounds checking, division by zero, etc. -- but they don't really apply to this case. 

## Compiling and Importing

The example above showed how to write the syntax for Cython files, with Jupyter taking care compiling and running the code. In practice, we'll need to be able to compile Cython files ourselves and be prepared for the pain that comes with sharing compiled code across platforms. 

There are a few ways to compile Cython programs, but the recommended method is to use Python's distutils functionality with a "setup.py" file. Suppose the program we want to compile is located in <a href="pyfiles/integrate.pyx" target="_blank">integrate.pyx</a>:

In [ ]:
# Standalone version of the demo integration function.

import cython
from libc.math cimport sin, cos

# Declare as a C function (cdef)
cdef double f(double x) except*:  # Add types to function signature
    return sin(x)*cos(x)

# Declare as a C/python function (cpdef)
@cython.cdivision(True)  # Remove error checking on python division
cpdef double integrate(double a, double b, int N) except*:  # Add types to function signature
    cdef int i  # Declare using "cdef"
    cdef double s, dx  # Declare using "cdef"
    
    s = 0
    dx = (b-a)/N
    for i in range(N):
        s += f(a+i*dx)
    return s * dx


To compile, write a <a href="pyfiles/setup.py" target="_blank">setup.py</a> file as below:

In [ ]:
from distutils.core import setup
from Cython.Build import cythonize

setup(
    name = 'Cython Demo',
    ext_modules = cythonize("integrate.pyx")
    )


Then, the program is compiled by typing ```python setup.py build_ext --inplace``` at the command line. Once compilation is complete, the ```integrate``` module can be imported as usual, as shown by the <a href="pyfiles/demo.py" target="_blank">demo.py</a> script below:

In [ ]:
import integrate
import numpy as np

a = 0
b = np.pi/2
N = 500000

x = integrate.integrate(a, b, N) # Should return 0.5

print('Cython integration returns %.4f (the answer is %.4f)' % (x, 0.5))
